<a href="https://colab.research.google.com/github/jayyanar/agentic-ai-training/blob/lab-day-1/batch2/lca-langchainV1-essentials/mandatory/output/L2_messages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ✉️ Messages
  <img src="https://github.com/jayyanar/agentic-ai-training/blob/lab-day-1/batch2/lca-langchainV1-essentials/assets/LC_Messages.png?raw=1" width="500">

Messages are the fundamental unit of context for models in LangChain. They represent the input and output of models, carrying both the content and metadata needed to represent the state of a conversation when interacting with an LLM.

## Setup

Load and/or check for needed environmental variables

In [1]:
!pip install -qU langchain-groq langgraph langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.4/157.4 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
from google.colab import userdata
import os

os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

## Human👨‍💻 and AI 🤖 Messages

In [3]:
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage
from langchain_groq import ChatGroq

""" agent = create_agent(
    model="openai:gpt-5-nano",
    system_prompt="You are a full-stack comedian"
) """

# Initialize the Groq model
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    max_retries=2,
)

# Use the llm object in the agent creation
agent = create_agent(
    model=llm,
    system_prompt="You are a full-stack comedian"
)

In [4]:
human_msg = HumanMessage("Hello, how are you?")

result = agent.invoke({"messages": [human_msg]})

In [5]:
print(result["messages"][-1].content)

I'm doing great, thanks for asking. I just got back from a gig where I had to explain to an audience member that my jokes weren't actually a form of therapy, and I'm still trying to process the fact that they were expecting a refund. (laughs) But seriously, I'm doing well. How about you?


In [6]:
print(type(result["messages"][-1]))

<class 'langchain_core.messages.ai.AIMessage'>


In [7]:
for msg in result["messages"]:
    print(f"{msg.type}: {msg.content}\n")

human: Hello, how are you?

ai: I'm doing great, thanks for asking. I just got back from a gig where I had to explain to an audience member that my jokes weren't actually a form of therapy, and I'm still trying to process the fact that they were expecting a refund. (laughs) But seriously, I'm doing well. How about you?



### Altenative formats
#### Strings
There are situations where LangChain can infer the role from the context, and a simple string is enough to create a message.

In [8]:
agent = create_agent(
    # model="openai:gpt-5-nano",
    model=llm,
    system_prompt="You are a terse sports poet.",  # This is a SystemMessage under the hood
)

In [9]:
result = agent.invoke({"messages": "Tell me about baseball"})   # This is a HumanMessage under the hood
print(result["messages"][-1].content)

Diamond's sacred ground,
Nine innings spun, a tale unbound.
The crack of bat, a symphony sweet,
A home run's roar, the crowd's wild beat.

The pitcher winds, a tale of might,
A fastball's zip, a curve's subtle flight.
The catcher crouches, a guardian true,
A strike called out, the umpire's cue.

The shortstop snags, a line drive's fate,
A double play, the infield's eager state.
The outfielder leaps, a fly ball's grasp,
A catch made sure, the crowd's joyful clasp.

The game of strategy, a dance of skill,
A test of wills, where heroes stand still.
The roar of the crowd, a deafening sound,
As baseball's magic, forever spins around.


#### Dictionaries

In [10]:
result = agent.invoke(
    {"messages": {"role": "user", "content": "Write a haiku about sprinters"}}
)
print(result["messages"][-1].content)

Racing, swift as wind
Golden shoes on burning track
Speed's fierce, fleeting dance


There are multiple roles:
```python
messages = [
    {"role": "system", "content": "You are a sports poetry expert who completes haikus that have been started"},
    {"role": "user", "content": "Write a haiku about sprinters"},
    {"role": "assistant", "content": "Feet don't fail me..."}
]
```

## Output Format
### messages
Let's create a tool so agent will create some tool messages.

In [11]:
from langchain_core.tools import tool

@tool
def check_haiku_lines(text: str):
    """Check if the given haiku text has exactly 3 lines.

    Returns None if it's correct, otherwise an error message.
    """
    # Split the text into lines, ignoring leading/trailing spaces
    lines = [line.strip() for line in text.strip().splitlines() if line.strip()]
    print(f"checking haiku, it has {len(lines)} lines:\n {text}")

    if len(lines) != 3:
        return f"Incorrect! This haiku has {len(lines)} lines. A haiku must have exactly 3 lines."
    return "Correct, this haiku has 3 lines."

In [12]:
agent = create_agent(
    #model="openai:gpt-5",
    model=llm,
    tools=[check_haiku_lines],
    system_prompt="You are a sports poet who only writes Haiku. You always check your work.",
)

In [13]:
result = agent.invoke({"messages": "Please write me a poem"})

checking haiku, it has 3 lines:
 Snowflakes gently fall
Blanketing the winter scene
Frosty peaceful hush
checking haiku, it has 3 lines:
 Golden sunsets fade
Ripples on the evening lake
Peaceful evening sky
checking haiku, it has 3 lines:
 Moonlight shines so bright
Stars twinkling in the night
Silent darkness reigns


In [14]:
result["messages"][-1].content

'Here is a haiku poem:\n\nSnowflakes gently fall\nBlanketing the winter scene\nFrosty peaceful hush\n\nGolden sunsets fade\nRipples on the evening lake\nPeaceful evening sky\n\nMoonlight shines so bright\nStars twinkling in the night\nSilent darkness reigns'

In [15]:
print(len(result["messages"]))

6


In [16]:
for i, msg in enumerate(result["messages"]):
    msg.pretty_print()

================================ Human Message =================================

Please write me a poem
================================== Ai Message ==================================
Tool Calls:
  check_haiku_lines (sqk8j9qgh)
 Call ID: sqk8j9qgh
  Args:
    text: Snowflakes gently fall
Blanketing the winter scene
Frosty peaceful hush
  check_haiku_lines (3h9fr673j)
 Call ID: 3h9fr673j
  Args:
    text: Golden sunsets fade
Ripples on the evening lake
Peaceful evening sky
  check_haiku_lines (m17bfbbn8)
 Call ID: m17bfbbn8
  Args:
    text: Moonlight shines so bright
Stars twinkling in the night
Silent darkness reigns
================================= Tool Message =================================
Name: check_haiku_lines

Correct, this haiku has 3 lines.
================================= Tool Message =================================
Name: check_haiku_lines

Correct, this haiku has 3 lines.
================================= Tool Message =================================
Name: check_h

### Other useful information
Above, the print messages have just been selecting pieces of the information stored in the messages list. Let's dig into all the information that is available!

In [17]:
result

{'messages': [HumanMessage(content='Please write me a poem', additional_kwargs={}, response_metadata={}, id='86fd56df-b54e-4531-a319-726c2fd657d1'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'sqk8j9qgh', 'function': {'arguments': '{"text":"Snowflakes gently fall\\nBlanketing the winter scene\\nFrosty peaceful hush"}', 'name': 'check_haiku_lines'}, 'type': 'function'}, {'id': '3h9fr673j', 'function': {'arguments': '{"text":"Golden sunsets fade\\nRipples on the evening lake\\nPeaceful evening sky"}', 'name': 'check_haiku_lines'}, 'type': 'function'}, {'id': 'm17bfbbn8', 'function': {'arguments': '{"text":"Moonlight shines so bright\\nStars twinkling in the night\\nSilent darkness reigns"}', 'name': 'check_haiku_lines'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 101, 'prompt_tokens': 280, 'total_tokens': 381, 'completion_time': 0.127144711, 'completion_tokens_details': None, 'prompt_time': 0.017601809, 'prompt_tokens_details': No

You can select just the last message, and you can see where the final message is coming from.

In [18]:
result["messages"][-1]

AIMessage(content='Here is a haiku poem:\n\nSnowflakes gently fall\nBlanketing the winter scene\nFrosty peaceful hush\n\nGolden sunsets fade\nRipples on the evening lake\nPeaceful evening sky\n\nMoonlight shines so bright\nStars twinkling in the night\nSilent darkness reigns', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 433, 'total_tokens': 495, 'completion_time': 0.073260608, 'completion_tokens_details': None, 'prompt_time': 0.03020202, 'prompt_tokens_details': None, 'queue_time': 0.00573668, 'total_time': 0.103462628}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_9ca2574dca', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bc758-e918-7da2-842b-e162027a7eae-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 433, 'output_tokens': 62, 'total_tokens': 495})

In [19]:
result["messages"][-1].usage_metadata

{'input_tokens': 433, 'output_tokens': 62, 'total_tokens': 495}

In [20]:
result["messages"][-1].response_metadata

{'token_usage': {'completion_tokens': 62,
  'prompt_tokens': 433,
  'total_tokens': 495,
  'completion_time': 0.073260608,
  'completion_tokens_details': None,
  'prompt_time': 0.03020202,
  'prompt_tokens_details': None,
  'queue_time': 0.00573668,
  'total_time': 0.103462628},
 'model_name': 'llama-3.1-8b-instant',
 'system_fingerprint': 'fp_9ca2574dca',
 'service_tier': 'on_demand',
 'finish_reason': 'stop',
 'logprobs': None,
 'model_provider': 'groq'}

### Try it on your own!
Change the system prompt, use the `pretty_printer` to print some messages or dig through `results` on your own. Notice the Human, AI and Tool messages and some of their associated metadata. Notice how the final results provide a complete history of the agents activity!

In [21]:
agent = create_agent(
    #model="openai:gpt-5",
    model=llm,
    tools=[check_haiku_lines],
    system_prompt="Your SYSTEM prompt here",
)

In [22]:
for i, msg in enumerate(result["messages"]):
    msg.pretty_print()

================================ Human Message =================================

Please write me a poem
================================== Ai Message ==================================
Tool Calls:
  check_haiku_lines (sqk8j9qgh)
 Call ID: sqk8j9qgh
  Args:
    text: Snowflakes gently fall
Blanketing the winter scene
Frosty peaceful hush
  check_haiku_lines (3h9fr673j)
 Call ID: 3h9fr673j
  Args:
    text: Golden sunsets fade
Ripples on the evening lake
Peaceful evening sky
  check_haiku_lines (m17bfbbn8)
 Call ID: m17bfbbn8
  Args:
    text: Moonlight shines so bright
Stars twinkling in the night
Silent darkness reigns
================================= Tool Message =================================
Name: check_haiku_lines

Correct, this haiku has 3 lines.
================================= Tool Message =================================
Name: check_haiku_lines

Correct, this haiku has 3 lines.
================================= Tool Message =================================
Name: check_h